In [1]:
import psycopg2

In [2]:
dir(psycopg2)

['BINARY',
 'Binary',
 'DATETIME',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'ROWID',
 'STRING',
 'Time',
 'TimeFromTicks',
 'Timestamp',
 'TimestampFromTicks',
 'Warning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__libpq_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_connect',
 '_ext',
 '_json',
 '_psycopg',
 '_range',
 'apilevel',
 'compat',
 'connect',
 'errors',
 'extensions',
 'paramstyle',
 'threadsafety',
 'tz']

In [4]:
pg_conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

In [5]:
pg_conn

<connection object at 0x7f6376fbc2b8; dsn: 'user=xmptgyod password=xxx dbname=xmptgyod host=raja.db.elephantsql.com', closed: 0>

In [6]:
pg_curs = pg_conn.cursor()

In [7]:
pg_curs.execute('SELECT * FROM test_table;')
pg_curs.fetchall()

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

In [10]:
# Download rpg_db.sqlite3
!wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

# Rename
!mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/joseph/.wget-hsts'. HSTS will be disabled.
--2019-06-04 12:48:38--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-06-04 12:48:38--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/modul

In [12]:
import sqlite3

sl_conn = sqlite3.Connection("rpg_db.sqlite3")
sl_curs = sl_conn.cursor()

In [13]:
sl_curs.execute('SELECT COUNT(*) FROM charactercreator_character;').fetchall()

[(302,)]

In [ ]:
# Goal - an ETL pipeline
# Get the character data from SQLite to PostgreSQL

In [14]:
characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()

In [15]:
# Should still be 302
len(characters)

302

In [16]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [17]:
characters[-1]

(302, 'Aliquam n', 0, 0, 10, 1, 1, 1, 1)

In [18]:
# We have the characters in local Python object
# Specifically, a list of tuples
# Next - CREATE TABLE in PostgreSQL (make the character schema)

create_character_table = """
CREATE TABLE charactercreator_character (

    character_id SERIAL PRIMARY KEY,
    name VARCHAR(30),
    level INT,
    exp INT,
    hp INT,
    strength INT,
    intelligence INT,
    dexterity INT,
    wisdom INT
)
"""

In [19]:
pg_curs.execute(create_character_table)

In [20]:
# Remember, commit is a Connection class method
pg_conn.commit()

In [21]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [24]:
attempted_insert = """
INSERT INTO charactercreator_character
VALUES """ + str(characters[0])

attempted_insert

"\nINSERT INTO charactercreator_character\nVALUES (1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [25]:
pg_curs.execute(attempted_insert)

In [26]:
# Commit to update table
pg_conn.commit()

In [28]:
# Create a new cursor after commiting 
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [29]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [30]:
len(pg_characters)

1

In [32]:
# Insert the rest of the characters & commit
for character in characters[1:]:
    insert_character = """
    INSERT INTO charactercreator_character
    VALUES """ + str(character)
    pg_curs.execute(insert_character)

pg_conn.commit()

In [33]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [34]:
# Verify number of inserts
len(pg_characters)

302

In [35]:
# View tail observation
pg_characters[-1]

(302, 'Aliquam n', 0, 0, 10, 1, 1, 1, 1)

In [36]:
# Verify integrity of inserts
for character, pg_character in zip(characters, pg_characters):
    assert character == pg_character